In [1]:
import pandas as pd
from pathlib import Path
from tqdm import tqdm

In [2]:
# from process.water import water_process1

In [3]:
data_dir = Path.cwd().parents[2] / 'original_data/train'
rain = pd.read_csv(data_dir / 'rainfall' / 'data.csv')
rain_st = pd.read_csv(data_dir / 'rainfall' / 'stations.csv')
tide = pd.read_csv(data_dir / 'tidelevel' / 'data.csv')
tide_st = pd.read_csv(data_dir / 'tidelevel' / 'stations.csv')
water = pd.read_csv(data_dir / 'waterlevel' / 'data.csv')
water_st = pd.read_csv(data_dir / 'waterlevel' / 'stations.csv')
dam = pd.read_csv(data_dir / 'dam.csv')

In [4]:
import pandas as pd
import numpy as np
import gc
import math

def rain_process1(rain, rain_st):
    ###################
    # data.csv側の処理 #
    ###################

    ##### 全体でdrop_duplicates
    rain.drop_duplicates(inplace=True)
    
    ##### (date, statition, city)が重複している行 -> 
    #####     その行の中でfloatが最も含まれている行を採用
    # (date, station, city)でグループを作った時、2行以上ある場合は重複なのでそのindexだけ取り出す
    nunique_date_st_city = rain.groupby(['date', 'station', 'city']).nunique()
    nunique_date_st_city['max_count'] = nunique_date_st_city.max(axis=1)

    dup_date_st_city = nunique_date_st_city.query('max_count >= 2')[['max_count']]
    dup_date_st_city_idx =  dup_date_st_city.index
    # 重複した行それぞれに含まれる値でfloatである値をカウント
    dup_date_st_city_df = rain.set_index(['date', 'station', 'city']).loc[dup_date_st_city_idx]
    dup_date_st_city_df['num_count'] = dup_date_st_city_df.apply(lambda x: \
        24 - pd.to_numeric(x, errors='coerce').isnull().sum(),axis=1)
    # floatが最も多い1行だけをとり出してconcat
    concat_df = None
    for _, df in dup_date_st_city_df.groupby(['date', 'station', 'city']):
        df = df.sort_values('num_count', na_position='first', ascending=False).iloc[0:1, :]
        if concat_df is None:
            concat_df = df.copy()
        else:
            concat_df = pd.concat([concat_df, df], axis=0)
    concat_df.drop('num_count', inplace=True, axis=1)

    # 重複していない行たちとconcat
    unique_date_st_city = nunique_date_st_city.query('max_count == 1')
    unique_date_st_city =  unique_date_st_city.index
    unique_date_st_city_df = rain.set_index(['date', 'station', 'city']).loc[unique_date_st_city]
    rain = pd.concat([unique_date_st_city_df, concat_df])

    del nunique_date_st_city, dup_date_st_city, dup_date_st_city_df, \
        unique_date_st_city, unique_date_st_city_df, concat_df, df
    gc.collect()

    rain.reset_index(inplace=True)
    rain.sort_values(['date', 'station', 'city'], inplace=True)

    ##### (おそらく)同じstationである行の値をマージ
    # 観測日数が31日のstationは、そのstation名に(電)のついたものと同じstationと考えられるのでマージ
    for st in rain['station'].value_counts()[rain['station'].value_counts() == 31].index:
        st_ = st + '(電)'
        # 変更前後の２つのstationに含まれるデータに日付の重なりがなければマージ
        if len(rain.query('station in (@st, @st_)')) == rain.query('station in (@st, @st_)')['date'].nunique():
            rain.loc[rain['station'] == st, 'station'] = st_

    # station.csvには存在しないstationで、(国)をつけたものなら存在するものはおなじstationとしてマージ
    for st in set(rain['station'].unique()) - set(rain_st['観測所名称'].unique()):
        bool_ = rain_st['観測所名称'].str.contains(st)
        if (bool_).any():
            st_ = rain_st[bool_]['観測所名称'].iloc[-1]
            if f'{st}(国)' == st_:
                # 変更前後の２つのstationに含まれるデータに日付の重なりがなければマージ
                if len(rain.query('station in (@st, @st_)')) == rain.query('station in (@st, @st_)')['date'].nunique():
                    rain.loc[rain['station'] == st, 'station'] = st_
    
    ######################
    # station.csv側の処理 #
    ######################

    ##### station名に(砂防)が含まれているものは入力時使用も0であり、ないものとマージできる
    rain_st.loc[:, '観測所名称'] = rain_st['観測所名称'].str.replace(r'\(砂防\)', '')

    ###################
    # データベースを作成 #
    ###################
    
    # idに(station, city)を対応させたテーブルを作る
    keys = rain.groupby(['station', 'city']).count().index
    rain_db = pd.DataFrame(index=keys).reset_index()
    rain_db['id'] = range(len(rain_db))
    rain_db = rain_db.reindex(columns=['id', 'station', 'city'])

    # column名に変更を加える
    rain_st = rain_st.rename(columns={'観測所名称': 'station', '市町': 'city'})

    # station.csvのcityがnanのもののうち、data.csvから埋められるものは埋める
    for data in rain_st.iterrows(): # stationを一列ずつ取り出す
        city = data[1]['city'] # cityを取り出す
        if isinstance(city, float) and math.isnan(city): # そのcityがnanの時のみ
            st = data[1]['station']
            city = rain.query('station==@st')['city'].unique()[0] # data.csvからそのstationを検索してなんのcityかをみる
            rain_st.loc[(rain_st['station'] == st), 'city'] = city
    
    # data.csvの(station, city)をidに置き換える
    rain = rain_db.merge(rain, on=['station', 'city'], how='left')
    rain.drop(['station', 'city'], axis=1, inplace=True)

    rain_st = rain_db.merge(rain_st, on=['station', 'city'], how='left')
    rain_st['入力時使用'] = rain_st['入力時使用'].fillna(0.0)

    return rain, rain_st

In [5]:
rain_p1, rain_st_p1 = rain_process1(rain, rain_st)

/var/folders/s8/kn1s0g_s407d23sqsmvzcql80000gn/T/ipykernel_48385/2105682670.py:59: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  bool_ = rain_st['観測所名称'].str.contains(st)
/var/folders/s8/kn1s0g_s407d23sqsmvzcql80000gn/T/ipykernel_48385/2105682670.py:72: FutureWarning: The default value of regex will change from True to False in a future version.
  rain_st.loc[:, '観測所名称'] = rain_st['観測所名称'].str.replace(r'\(砂防\)', '')


In [6]:
rain_db01 = pd.read_csv(Path.cwd().parents[2] / 'data/database01/rain_data.csv')
rain_st_db01 = pd.read_csv(Path.cwd().parents[2] / 'data/database01/rain_station.csv')
print(rain_p1.equals(rain_db01))
print(rain_st_p1.equals(rain_st_db01))

True
True


In [18]:
def water_process1(water, water_st):

    water['station'] = water['station'].str.replace(r'\(電\)', '')

    for st in set(water['station'].unique()) - set(water_st['観測所名称'].unique()):
        bool_ = water_st['観測所名称'].str.contains(st)
        if (bool_).any():
            st_ = water_st[bool_]['観測所名称'].iloc[-1]
            if f'{st}(国)' == st_:
                if len(water.query('station in (@st, @st_)')) == water.query('station in (@st, @st_)')['date'].nunique():
                    water.loc[water['station'] == st, 'station'] = st_
    
    water.loc[water['station'] == '山手', 'station'] = '山手(国)'

    water_st.loc[:, '河川名'] = water_st['河川名'].str.replace('\\n', '')
    
    keys = water.groupby(['station', 'river']).count().index
    water_db = pd.DataFrame(index=keys).reset_index()
    water_db['id'] = range(len(water_db))
    water_db = water_db.reindex(columns=['id', 'station', 'river'])

    water = water_db.merge(water, on=['station', 'river'], how='left')
    water.drop(['station', 'river'], axis=1, inplace=True)
    
    water_st = water_st.rename(columns={'観測所名称': 'station', '河川名': 'river'})
    water_st = water_db.merge(water_st, on=['station', 'river'], how='left')

    water_st.loc[water_st['river']=='太田川放水路', 'river'] = '太田川\\n放水路' # 改行があるとうまく処理されなかったので最後に改行を付け直す

    return water, water_st

In [19]:
water_p1, water_st_p1 = water_process1(water, water_st)

/var/folders/s8/kn1s0g_s407d23sqsmvzcql80000gn/T/ipykernel_48385/4108981710.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  water['station'] = water['station'].str.replace(r'\(電\)', '')
/var/folders/s8/kn1s0g_s407d23sqsmvzcql80000gn/T/ipykernel_48385/4108981710.py:6: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  bool_ = water_st['観測所名称'].str.contains(st)
/var/folders/s8/kn1s0g_s407d23sqsmvzcql80000gn/T/ipykernel_48385/4108981710.py:15: FutureWarning: The default value of regex will change from True to False in a future version.
  water_st.loc[:, '河川名'] = water_st['河川名'].str.replace('\\n', '')


In [21]:
def tide_process1(tide, tide_st):
    tide.loc[tide['station'] == '柿浦漁港', 'station'] = '柿浦港'
    tide.loc[tide['station'] == '呉阿賀港', 'station'] = '呉(阿賀)港'
    tide.loc[tide['station'] == '倉橋漁港', 'station'] = '倉橋港'

    keys = tide.groupby(['station', 'city']).count().index
    tide_db = pd.DataFrame(index=keys).reset_index()
    tide_db['id'] = range(len(tide_db))
    tide_db = tide_db.reindex(columns=['id', 'station', 'city'])

    tide = tide_db.merge(tide, on=['station', 'city'], how='left')
    tide.drop(['station', 'city'], axis=1, inplace=True)

    tide_st = tide_st.rename(columns={'観測所名': 'station'})
    tide_st = tide_db.merge(tide_st, on=['station'], how='left')
    
    return tide, tide_st

In [22]:
tide_p1, tide_st_p1 = tide_process1(tide, tide_st)

In [24]:
def convert_timeseries(df):
    all_id = df['id'].unique()
    all_date = df['date'].unique().astype(int)
    n_ids = len(all_id)
    all_id.sort()
    all_date.sort()
    null_value = 'x'

    data = []
    for st in all_id:
        for date in all_date:
            data.append([date, st])
    
    old_df = df.copy()
    old_df = old_df[~old_df[['date', 'id']].duplicated()]
    old_df.fillna(null_value, inplace=True)

    new_df = pd.DataFrame(data, columns=['date', 'id'])
    new_df = pd.merge(new_df, old_df, on=['date', 'id'], how='left')
    new_df = new_df.sort_values(['id', 'date']).reset_index(drop=True)
    del old_df
    
    series = new_df.iloc[:, 2:].values.reshape(n_ids, -1).T
    timeseries_df = pd.DataFrame(series, columns=all_id)

    dates = []
    hours = []
    for date in all_date:
        for hour in range(24):
            dates.append(date)
            hours.append(hour)
    timeseries_df['date'] = dates
    timeseries_df['hour'] = hours
    timeseries_df = timeseries_df.reindex(columns=['date', 'hour'] + list(all_id))
    return timeseries_df

In [25]:
rain_t = convert_timeseries(rain_p1)
water_t = convert_timeseries(water_p1)
tide_t = convert_timeseries(tide_p1)